In [1]:
import xpress as xp
import numpy as np
import pandas as pd
import time

In [2]:
print("Loading Data")
# Load distance matrix
distance_matrix = np.loadtxt('distance_matrix_reduce.csv', delimiter=',')
stations = pd.read_csv('candidate_stations_P300_kmeans_no_snap.csv')
pois = pd.read_csv('reduced_pois.csv')

Loading Data


In [3]:
I = distance_matrix.shape[0]  # Number of POIs (demand points)
J = distance_matrix.shape[1] # Number of candidate stations

In [ ]:
# Parameters
COST_REOPEN_EXISTING = 50000    # £ to reopen existing station (B)
COST_BUILD_NEW = 150000         # £ to build new station (A)
BUDGET = 2000000                # £ total budget (C)

In [5]:
cost_vector = np.where(stations['is_existing_station'], COST_REOPEN_EXISTING, COST_BUILD_NEW)

In [6]:
print("DEFINE MODEL PARAMETERS")
# IMPORTANT: Set the number of facilities to locate
# p = 85  # Change this to test different scenarios

print(f"\n{'='*70}")
print(f"MODEL PARAMETERS")
print(f"{'='*70}")
print(f"Number of POIs (I):                {I}")
print(f"Number of candidate stations (J):  {J}")

DEFINE MODEL PARAMETERS

MODEL PARAMETERS
Number of POIs (I):                5926
Number of candidate stations (J):  379


In [7]:
print("CREATE XPRESS PROBLEM")
prob = xp.problem(name='p_centre_reduce')

CREATE XPRESS PROBLEM


/var/folders/tn/v19h6ld131n5p0bkj6b850sr0000gn/T/ipykernel_14365/926094487.py:2: LicenseWarning: Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
  prob = xp.problem(name='p_centre_reduce')


In [8]:
print("="*70)
print("DEFINE DECISION VARIABLES")
print("="*70)
# Y[j] = 1 if facility j is opened, 0 otherwise
Y = {j: prob.addVariable(vartype=xp.binary, name=f'Y_{j}') for j in range(J)}

# X[i,j] = 1 if demand point i is assigned to station j, 0 otherwise
X = {(i, j): prob.addVariable(vartype=xp.binary, name=f'X_{i}_{j}') 
     for i in range(I) for j in range(J)}

# Q = maximum service distance (minimax objective)
Q = prob.addVariable(name='Q', lb=0)

print(f"Created {J} Y variables (station selection)")
print(f" Created {I*J:,} X variables (POI-station assignments)")
print(f" Created 1 W variable (minimax distance)")
print(f"  Total variables: {J + I*J + 1:,}")


DEFINE DECISION VARIABLES
Created 379 Y variables (station selection)
 Created 2,245,954 X variables (POI-station assignments)
 Created 1 W variable (minimax distance)
  Total variables: 2,246,334


In [9]:
print("="*70)
print("ADD CONSTRAINTS")
print("="*70)
# CONSTRAINT 1: Each POI must be assigned to exactly one station
print("  Adding constraint 1: POI assignment (each POI assigned to exactly one station)...")
for i in range(I):
    prob.addConstraint(xp.Sum(X[i, j] for j in range(J)) == 1)

# CONSTRAINT 2: Exactly p stations must be opened
# print(f"  Adding constraint 2: Facility count (open exactly {p} stations)...")
# prob.addConstraint(xp.Sum(Y[j] for j in range(J)) == p)

# CONSTRAINT 3: Assignment only to open stations (linking constraints)
print("  Adding constraint 3: Linking constraints (assign only to open stations)...")
for i in range(I):
    for j in range(J):
        prob.addConstraint(X[i, j] <= Y[j])

# CONSTRAINT 4: Minimax distance constraint
print("  Adding constraint 4: Minimax distance (Q >= distance to assigned station)...")
for i in range(I):
    prob.addConstraint(
        xp.Sum(distance_matrix[i, j] * X[i, j] for j in range(J)) <= Q)


# CONSTRAINT 5: BUDGET CONSTRAINT (NEW - KEY ADDITION)
print(f"  Constraint 5: Budget constraint (£{BUDGET:,})...")
prob.addConstraint(xp.Sum(cost_vector[j] * Y[j] for j in range(J)) <= BUDGET)

ADD CONSTRAINTS
  Adding constraint 1: POI assignment (each POI assigned to exactly one station)...
  Adding constraint 3: Linking constraints (assign only to open stations)...
  Adding constraint 4: Minimax distance (Q >= distance to assigned station)...
  Constraint 5: Budget constraint (£2,000,000)...


In [ ]:
# #CONSTRAINT demand
# for j in range(J):
#         prob.addConstraint(Y[j]<= Z[j] <= C[j] * Y[j])

# for i in range(I):
#     prob.addConstraint(
#         xp.Sum(X[i, j] * demand[i] for j in range(J)) <= Z[j])

In [ ]:
print("="*70)
print("SET OBJECTIVE FUNCTION")
print("="*70)
# Minimize Q (the maximum service distance)
prob.setObjective(Q, sense=xp.minimize)

SET OBJECTIVE FUNCTION


: 

In [ ]:
print("\n" + "="*70)
print("SOLVING MODEL...")
print("="*70 + "\n")

start_solve = time.time()

# Solve the problem
xp.setOutputEnabled(True)
prob.solve()

solve_time = time.time() - start_solve


SOLVING MODEL...

FICO Xpress v9.7.0, Hyper, solve started 17:32:08, Nov 12, 2025
Heap usage: 1293MB (peak 1293MB, 295MB system)
Minimizing MILP p_centre_reduce using up to 8 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
   2257807 rows      2246334 cols      8990111 elements   2246333 entities
Presolved problem has:
   2257807 rows      2246334 cols      8868393 elements   2246333 entities
Presolve finished in 151 seconds
Heap usage: 2586MB (peak 3485MB, 295MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 4.60e-05,  1.50e+05] / [ 6.25e-07,  2.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  2.00e+06] / [ 9.30e-02,  5.30e+02]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 8.00e+00,  8.00e+00]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 6.4GB
 *

In [ ]:
    print("\n" + "="*70)
    print("SOLUTION SUMMARY")
    print("="*70)
    

    # Extract optimal minimax distance
    Q_optimal = prob.getSolution(Q)
    
    # Extract which stations to open
    stations_open = []
    for j in range(J):
        if prob.getSolution(Y[j]) > 0.5:  # Binary variable, so > 0.5 means 1
            stations_open.append(j)

    # Classify as existing or new
    existing_open = [j for j in stations_open if cost_vector[j] = 50000]
    new_open = [j for j in stations_open if cost_vector[j] = 150000]
    
    # Calculate total cost
    total_cost = sum(cost_vector[j] for j in stations_open)
    



SOLUTION SUMMARY


In [ ]:
    print(f"\n{'='*70}")
    print(f"OPTIMAL P-CENTRE RESULTS")
    print(f"{'='*70}")
    
    print(f"\n1. STATIONS TO OPEN ({len(stations_open)} out of {J}):")
    print(f"   Station indices: {stations_open}")
    
    print(f"\n   Station details:")
    optimal_stations_list = []
    for idx, j in enumerate(stations_open, 1):
        station_id = stations.iloc[j]['candidate_id']
        lat = station_lat = stations.iloc[j]['centroid_lat']        
        lon = station_lon = stations.iloc[j]['centroid_lon']
        
        optimal_stations_list.append({
            'index': j,
            'rank': idx,
            'station_id': station_id,
            'latitude': lat,
            'longitude': lon
        })
        
        print(f"   {idx:2d}. (Station ID = {station_id})")
    
    print(f"\n2. OPTIMAL MINIMAX DISTANCE:")
    print(f"   Maximum service distance (Q*) = {Q_optimal:.4f} km")
    print(f"   All {I:,} POIs served within {Q_optimal:.4f} km of nearest station")


OPTIMAL P-CENTRE RESULTS

1. STATIONS TO OPEN (85 out of 379):
   Station indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 73, 113, 119, 137, 151, 182, 197, 204, 221, 237, 252, 254, 255, 259, 278, 282, 294]

   Station details:
    1. (Station ID = 0)
    2. (Station ID = 1)
    3. (Station ID = 2)
    4. (Station ID = 3)
    5. (Station ID = 4)
    6. (Station ID = 5)
    7. (Station ID = 6)
    8. (Station ID = 7)
    9. (Station ID = 8)
   10. (Station ID = 9)
   11. (Station ID = 10)
   12. (Station ID = 11)
   13. (Station ID = 12)
   14. (Station ID = 13)
   15. (Station ID = 16)
   16. (Station ID = 17)
   17. (Station ID = 18)
   18. (Station ID = 19)
   19. (Station ID = 20)
   20. (Station ID = 21)
   21. (Station ID = 22)
   22. (Station ID = 23)
   23. (

In [ ]:
    sol = {(i, j): prob.getSolution(X[i, j]) for i in range(I) for j in range(J)}
        solution_matrix = np.zeros((I, J))
        for (i, j), val in sol.items():
            solution_matrix[i, j] = val
        assignments = np.argmax(solution_matrix, axis=1)

# Calculate metrics
    total_demand = demand.sum()
    demand_satisfied = sum(demand[assignments == j] for j in stations_open)
    
    selected_capacity = sum(stations.iloc[j]['capacity'] for j in stations_open)
    avg_utilization = 100 * demand_satisfied / selected_capacity if selected_capacity > 0 else 0
    
    all_distances = [distance_matrix[i, assignments[i]] for i in range(I)]
    mean_distance = np.mean(all_distances)

IndentationError: unexpected indent (3341590220.py, line 2)

In [ ]:
print(f"\n1. OPTIMAL MINIMAX DISTANCE:")
    print(f"   W* = {Q_optimal:.4f} km")
    
    print(f"\n2. STATIONS OPENED: {len(stations_open)} total")
    print(f"   ├─ Existing (reopen): {len(existing_open)} stations")
    print(f"   └─ New (build): {len(new_open)} stations")
    
    print(f"\n3. COST BREAKDOWN:")
    existing_cost = sum(cost_vector[j] for j in existing_open)
    new_cost = sum(cost_vector[j] for j in new_open)
    print(f"   Reopen existing: {len(existing_open)} × £{COST_REOPEN_EXISTING:,} = £{existing_cost:,}")
    print(f"   Build new:      {len(new_open)} × £{COST_BUILD_NEW:,} = £{new_cost:,}")
    print(f"   Total cost:     £{total_cost:,}")
    print(f"   Budget used:    {100*total_cost/BUDGET:.1f}% of £{BUDGET:,}")
    print(f"   Remaining:      £{BUDGET - total_cost:,}")
    
    
    print(f"\n4. STATIONS TO REOPEN (Existing):")
    for j in sorted(existing_open):
        cost = cost_vector[j]
        pois_count = (assignments == j).sum()
        print(f"   [{j:2d}] {stations.iloc[j]['name']:30s} | Cost: £{cost:,} | Serves {pois_count:,} POIs")
    
    print(f"\n6. STATIONS TO BUILD (New):")
    for j in sorted(new_open):
        cost = cost_vector[j]
        pois_count = (assignments == j).sum()
        lat, lon = stations.iloc[j]['lat'], stations.iloc[j]['lon']
        print(f"   [{j:2d}] Location ({lat:.4f}, {lon:.4f}) | Cost: £{cost:,} | Serves {pois_count:,} POIs")